In [ ]:
import shutil
import os
files_to_move = ['util_functions.py', 'data_utils.py', 'preprocessing.py']
for f in files_to_move:
  if not os.path.exists(f):
    shutil.move(os.path.join('IGMC', f), f)

In [7]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch.optim import Adam
from torch_geometric.data import DataLoader
from torch_geometric.nn import RGCNConv
from torch_geometric.utils import dropout_adj
from IGMC.util_functions import *
from IGMC.data_utils import *
from IGMC.preprocessing import *

In [8]:
# Arguments
EPOCHS=2
BATCH_SIZE=50
LR=1e-3
LR_DECAY_STEP = 20
LR_DECAY_VALUE = 10

torch.manual_seed(123)
device = torch.device('cpu')
(u_features, v_features, adj_train, train_labels, train_u_indices, train_v_indices, val_labels, 
val_u_indices, val_v_indices, test_labels, test_u_indices, test_v_indices, class_values
) = load_official_trainvaltest_split('ml_100k', testing=True)

User features shape: (943, 23)
Item features shape: (1682, 18)


In [21]:
train_dataset = eval('MyDynamicDataset')(root='data/ml_100k/testmode/train', A=adj_train, 
    links=(train_u_indices, train_v_indices), labels=train_labels, h=1, sample_ratio=1.0, 
    max_nodes_per_hop=200, u_features=None, v_features=None, class_values=class_values)
test_dataset = eval('MyDataset')(root='data/ml_100k/testmode/test', A=adj_train, 
    links=(test_u_indices, test_v_indices), labels=test_labels, h=1, sample_ratio=1.0, 
    max_nodes_per_hop=200, u_features=None, v_features=None, class_values=class_values)

len(train_dataset), len(test_dataset)

(80000, 20000)

In [10]:
class IGMC(torch.nn.Module):
    def __init__(self):
        super(IGMC, self).__init__()
        self.rel_graph_convs = torch.nn.ModuleList()
        self.rel_graph_convs.append(RGCNConv(in_channels=4, out_channels=32, num_relations=5, num_bases=4))
        self.rel_graph_convs.append(RGCNConv(in_channels=32, out_channels=32, num_relations=5, num_bases=4))
        self.rel_graph_convs.append(RGCNConv(in_channels=32, out_channels=32, num_relations=5, num_bases=4))
        self.rel_graph_convs.append(RGCNConv(in_channels=32, out_channels=32, num_relations=5, num_bases=4))
        self.linear_layer1 = Linear(256, 128)
        self.linear_layer2 = Linear(128, 1)

    def reset_parameters(self):
        self.linear_layer1.reset_parameters()
        self.linear_layer2.reset_parameters()
        for i in self.rel_graph_convs:
            i.reset_parameters()

    def forward(self, data):
        num_nodes = len(data.x)
        edge_index_dr, edge_type_dr = dropout_adj(data.edge_index, data.edge_type, p=0.2, num_nodes=num_nodes, training=self.training)

        out = data.x
        h = []
        for conv in self.rel_graph_convs:
            out = conv(out, edge_index_dr, edge_type_dr)
            out = torch.tanh(out)
            h.append(out)
        h = torch.cat(h, 1)
        h = [h[data.x[:, 0] == True], h[data.x[:, 1] == True]]
        g = torch.cat(h, 1)
        out = self.linear_layer1(g)
        out = F.relu(out)
        out = F.dropout(out, p=0.5, training=self.training)
        out = self.linear_layer2(out)
        out = out[:,0]
        return out

model = IGMC()

In [11]:
train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False, num_workers=2)

/Users/user/Documents/Nauka/Studia/Magisterskie/Magisterka/Recommender/.venv/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [12]:
model.to(device)
model.reset_parameters()
optimizer = Adam(model.parameters(), lr=LR, weight_decay=0)

In [14]:
batches_per_epoch = len(train_loader)
for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss_all = 0
    for i, train_batch in enumerate(train_loader):
        print(f"{i}/{batches_per_epoch}")
        optimizer.zero_grad()
        train_batch = train_batch.to(device)
        y_pred = model(train_batch)
        y_true = train_batch.y
        train_loss = F.mse_loss(y_pred, y_true)
        train_loss.backward()
        train_loss_all += BATCH_SIZE * float(train_loss)
        optimizer.step()
        torch.cuda.empty_cache()
    train_loss_all = train_loss_all / len(train_loader.dataset)
    
    print('epoch', epoch,'; train loss', train_loss_all)

    if epoch % LR_DECAY_STEP == 0:
      for param_group in optimizer.param_groups:
          param_group['lr'] = param_group['lr'] / LR_DECAY_VALUE

0/1600


/Users/user/Documents/Nauka/Studia/Magisterskie/Magisterka/Recommender/.venv/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


1/1600
2/1600
3/1600
4/1600
5/1600
6/1600
7/1600
8/1600
9/1600
10/1600
11/1600
12/1600
13/1600
14/1600
15/1600
16/1600
17/1600
18/1600
19/1600
20/1600
21/1600
22/1600
23/1600
24/1600
25/1600
26/1600
27/1600
28/1600
29/1600
30/1600
31/1600
32/1600
33/1600
34/1600
35/1600
36/1600
37/1600
38/1600
39/1600
40/1600
41/1600
42/1600
43/1600
44/1600
45/1600
46/1600
47/1600
48/1600
49/1600
50/1600
51/1600
52/1600
53/1600
54/1600
55/1600
56/1600
57/1600
58/1600
59/1600
60/1600
61/1600
62/1600
63/1600
64/1600
65/1600
66/1600
67/1600
68/1600
69/1600
70/1600
71/1600
72/1600
73/1600
74/1600
75/1600
76/1600
77/1600
78/1600
79/1600
80/1600
81/1600
82/1600
83/1600
84/1600
85/1600
86/1600
87/1600
88/1600
89/1600
90/1600
91/1600
92/1600
93/1600
94/1600
95/1600
96/1600
97/1600
98/1600
99/1600
100/1600
101/1600
102/1600
103/1600
104/1600
105/1600
106/1600
107/1600
108/1600
109/1600
110/1600
111/1600
112/1600
113/1600
114/1600
115/1600
116/1600
117/1600
118/1600
119/1600
120/1600
121/1600
122/1600
123/1600
1

In [15]:
batches_in_eval = len(test_loader)
model.eval()
test_loss = 0
for i, test_batch in enumerate(test_loader):
    print(f"{i}/{batches_in_eval}")
    test_batch = test_batch.to(device)
    with torch.no_grad():
        y_pred = model(test_batch)
    y_true = test_batch.y
    test_loss += F.mse_loss(y_pred, y_true, reduction='sum')
    # torch.cuda.empty_cache()
mse_loss = float(test_loss) / len(test_loader.dataset)

print('test MSE loss', mse_loss)
print('test RMSE loss', math.sqrt(mse_loss))

0/400


/Users/user/Documents/Nauka/Studia/Magisterskie/Magisterka/Recommender/.venv/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


1/400
2/400
3/400
4/400
5/400
6/400
7/400
8/400
9/400
10/400
11/400
12/400
13/400
14/400
15/400
16/400
17/400
18/400
19/400
20/400
21/400
22/400
23/400
24/400
25/400
26/400
27/400
28/400
29/400
30/400
31/400
32/400
33/400
34/400
35/400
36/400
37/400
38/400
39/400
40/400
41/400
42/400
43/400
44/400
45/400
46/400
47/400
48/400
49/400
50/400
51/400
52/400
53/400
54/400
55/400
56/400
57/400
58/400
59/400
60/400
61/400
62/400
63/400
64/400
65/400
66/400
67/400
68/400
69/400
70/400
71/400
72/400
73/400
74/400
75/400
76/400
77/400
78/400
79/400
80/400
81/400
82/400
83/400
84/400
85/400
86/400
87/400
88/400
89/400
90/400
91/400
92/400
93/400
94/400
95/400
96/400
97/400
98/400
99/400
100/400
101/400
102/400
103/400
104/400
105/400
106/400
107/400
108/400
109/400
110/400
111/400
112/400
113/400
114/400
115/400
116/400
117/400
118/400
119/400
120/400
121/400
122/400
123/400
124/400
125/400
126/400
127/400
128/400
129/400
130/400
131/400
132/400
133/400
134/400
135/400
136/400
137/400
138/400
139/

In [18]:
for i, test_batch in enumerate(test_loader):
    print(f"{i}/{batches_in_eval}")
    test_batch = test_batch.to(device)
    with torch.no_grad():
        y_pred = model(test_batch)
    y_true = test_batch.y
    print(y_pred, y_true, test_batch)
    break

0/400


/Users/user/Documents/Nauka/Studia/Magisterskie/Magisterka/Recommender/.venv/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


tensor([2.8263, 3.7069, 2.6431, 3.4155, 3.6947, 3.4965, 3.0314, 4.0038, 3.4828,
        3.9376, 4.2406, 3.7800, 4.3791, 4.4386, 2.9443, 3.0230, 2.7087, 3.6991,
        3.1125, 3.0586, 3.6640, 3.5894, 3.9088, 2.1263, 2.8193, 2.8586, 2.5074,
        2.8876, 2.9754, 2.7266, 3.0391, 3.5461, 2.9779, 3.1280, 2.3354, 3.0939,
        3.3424, 3.6943, 3.9164, 3.1196, 4.4169, 3.1951, 3.0381, 3.5352, 3.6714,
        3.3282, 3.7901, 4.0169, 3.2893, 3.9721]) tensor([2.8263, 3.7069, 2.6431, 3.4155, 3.6947, 3.4965, 3.0314, 4.0038, 3.4828,
        3.9376, 4.2406, 3.7800, 4.3791, 4.4386, 2.9443, 3.0230, 2.7087, 3.6991,
        3.1125, 3.0586, 3.6640, 3.5894, 3.9088, 2.1263, 2.8193, 2.8586, 2.5074,
        2.8876, 2.9754, 2.7266, 3.0391, 3.5461, 2.9779, 3.1280, 2.3354, 3.0939,
        3.3424, 3.6943, 3.9164, 3.1196, 4.4169, 3.1951, 3.0381, 3.5352, 3.6714,
        3.3282, 3.7901, 4.0169, 3.2893, 3.9721]) DataBatch(x=[14116, 4], edge_index=[2, 541752], y=[50], edge_type=[541752], batch=[14116], ptr=[51])
